In [0]:
!pip install tweet-preprocessor
!pip install google-api-python-client
import pandas as pd
from sklearn import preprocessing
import pandas as pd
import preprocessor as p
import networkx as nx
import matplotlib.pyplot as plt
import tweepy
import json
import json
import requests
import csv
import re
import sys
import urllib.parse

from sklearn.feature_extraction.text import CountVectorizer

from googleapiclient.discovery import build
from datetime import datetime

df = pd.read_csv("/content/drive/My Drive/news/news.csv", sep=',')
df.sort_values(by=['DATE'], inplace=True,ignore_index=True)
df.head()

In [0]:
import requests
from bs4 import BeautifulSoup
import spacy
ner = spacy.load("en_core_web_sm")
def getArticle(url):
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c)

    article_text = ''
    article = soup.findAll('p')
    for element in article:
        article_text += '\n' + ''.join(element.findAll(text = True))
    return article_text

def getEntity(article_cor):
  ent = ner(article_cor)
  entities = []
  for entity in ent.ents:
    entities.append(entity.text)
  return entities

def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      
                   vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                       reverse=True)
    return words_freq[:n]
def getTopicWords(corpus):
    topic = ""
    topics = get_top_n_words(corpus,5)
    for item in topics:
      topic = topic + item[0] + " "
    return topic.strip()

def getTopics(url):
    topic = ""
    article_text = getArticle(url)
    articleEntity = getEntity(article_text)
    topics = get_top_n_words(articleEntity,5)
    for item in topics:
      topic = topic + item[0] + " "
    return topic.strip()


In [0]:
# le = preprocessing.LabelEncoder()
# df['DATE_TRANS'] = le.fit_transform(df['DATE'])

In [0]:
df["ARTICLES"] = df["URL"].apply(lambda x: getArticle(x))
df.head()

In [0]:
df = df.drop([5,20,21,25,26])

In [0]:
df["TOPICS"] = df["ARTICLES"].apply(lambda x: getTopicWords([x]))
df.head()

In [0]:
df.tail()

In [0]:
date = []
topic = []
for index, row in df.iterrows():
  tempdate = [row["DATE"]]
  date.extend(tempdate*5)
  topic.extend(row["TOPICS"].split(" "))
print(len(date))
print(len(topic))

In [0]:
le = preprocessing.LabelEncoder()
df_news = pd.DataFrame() 
df_news["source"] = le.fit_transform(date)
df_news["target"] = le.fit_transform(topic)
df_news.tail()

In [0]:
df_news.to_csv("embeddings.csv")

In [0]:
import networkx as nx
df_news['label'] = 'topic'
Gnx = nx.from_pandas_edgelist(df_news, edge_attr='label')
nx.set_node_attributes(Gnx, 'news', 'label')

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
headline = df["HEADLINE"]
headline = [i.replace("‘","") for i in headline]
headline = [i.replace("’","") for i in headline]
headline = [i.replace("'","") for i in headline]
headline = [i.replace("?","") for i in headline]
headline = [i.replace(",","") for i in headline]
headline = [i.replace("(","") for i in headline]
headline = [i.replace(")","") for i in headline]
headline = [i.replace("-","") for i in headline]
headline = [i.split(" ") for i in headline]
# headline = np.array(headline) 
print(headline)
# headline_arr = np.array(headline)

# for item in headline_arr:
#   item = np.array(item)
# print(headline_arr)

In [0]:
# df_encoder = pd.DataFrame() 
# label_encoder =LabelEncoder()
# headline_int = label_encoder.fit_transform(headline).reshape(*headline.shape)
# one_hot_encoder = OneHotEncoder.fit_transform(headline_int).toarray()
# df_encoder = one_hot_encoder
# df_encoder.to_csv("news.content")

In [0]:
feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names =  feature_names + ['subject']
node_data = pd.read_csv("/content/drive/My Drive/cora/cora.content", sep='\t', header=None, names=column_names)
node_data.head()
node_features = node_data[feature_names]
node_features = node_features[:91]


In [0]:
node_features.reset_index(drop=True, inplace=True)
node_features.head()

In [0]:
len(node_features)

In [0]:
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.0.0rc1

In [0]:
import networkx as nx
import random
import numpy as np
import pandas as pd
import keras
import stellargraph as sg
try:
    sg.utils.validate_notebook_version("1.0.0rc1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.0.0rc1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None
from stellargraph.mapper import GraphSAGELinkGenerator, GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UnsupervisedSampler


In [0]:
# G = sg.StellarGraph(Gnx)
G = sg.StellarGraph(Gnx, node_features=node_features)

In [0]:
print(G.info())

In [0]:
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding


from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
%matplotlib inline
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True
)

In [0]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=True
)

In [0]:
epochs = 50
train_gen = FullBatchLinkGenerator(G_train, method="gcn")
train_flow = train_gen.flow(edge_ids_train, edge_labels_train)

In [0]:
test_gen = FullBatchLinkGenerator(G_test, method="gcn")
test_flow = train_gen.flow(edge_ids_test, edge_labels_test)

In [0]:
gcn = GCN(
    layer_sizes=[16, 16], activations=["relu", "relu"], generator=train_gen, dropout=0.3
)

In [0]:
x_inp, x_out = gcn.in_out_tensors()

In [0]:
prediction = LinkEmbedding(activation="relu", method="ip")(x_out)

In [0]:
prediction = keras.layers.Reshape((-1,))(prediction)

In [0]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.01),
    loss=keras.losses.binary_crossentropy,
    metrics=["acc"],
)

In [0]:
init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [0]:
history = model.fit(
    train_flow, epochs=epochs, validation_data=test_flow, verbose=2, shuffle=False
)

In [0]:
sg.utils.plot_history(history)

In [0]:
train_metrics = model.evaluate(train_flow)
test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [0]:
# predcit1 = train_gen.flow([[12,48]],[1])
# prediction1 = model.predict(predcit1)
# prediction1

In [0]:
data = ""
lastTopic = ""
for index, row in df.iterrows():
  data = data + row["TOPICS"] + "\n"
  lastTopic = row["TOPICS"]
print(data.strip())
topiclist = lastTopic.split(" ")
print(topiclist)

In [0]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# generate a sequence from a model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

# source text
# prepare the tokenizer on the source text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# create line-based sequences
sequences = list()
for line in data.split('\n'):
	encoded = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(encoded)):
		sequence = encoded[:i+1]
		sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# pad input sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=2)
# evaluate model
predcitTopics = []
for word in topiclist:
   predcitTopics.append(generate_seq(model, tokenizer, max_length-1, word, 4))

print(predcitTopics)